# **Exploratory Data Analysis (EDA) : Active interest rates by type of credit in Colombia**

## **1. Introduction**

* **Objective**: 
  The main objective of this notebook is explore and analyze the behavior of active interest rates by type of credit in Colombia using the available dataset.

* **Dataset Description**:
    The dataset corresponds to the weighted average interest rates, the amount and number of loans disbursed during the week for different types of credit (consumer, housing, ordinary commercial, preferential or corporate commercial, treasury commercial, special commercial and productive loans), classified by type of person, sex, company size, type of loan, type of guarantee, loan product, loan term, ethnic group, company age, type of rate, range by amount disbursed in current monthly legal minimum wages (SMLMV), debtor class, CIIU code and municipality code, of the credit establishments. The dataset is provided by the **Superintendencia Financiera de Colombia (SFC)** and hosted by **Datos Abiertos Colombia** platform. The data is updated frequently and can be accessed [here](https://www.datos.gov.co/Econom-a-y-Finanzas/Tasas-de-Inter-s-Activas-por-tipo-de-cr-dito/3y7m-j5ua).

* **Date Range**: 
    The dataset covers data from october 10 of 20225 to the most recent available date (december 2025).

* **Tools and Libraries**:
    The analysis will be conducted using Python programming language with libraries such as 
    - DuckDB
    - Pandas
    - NumPy 
    - Matplotlib and Seaborn 



## **2. Data Loading**

* **Import Libraries**: 
    First, we need to import the necessary libraries for data manipulation and visualization.

In [2]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv

* **Config the data source**

In [3]:
BASE_DIR = Path.cwd().parent
load_dotenv(BASE_DIR / '.env')
DATA_DIR = BASE_DIR / 'data'
DATA_URL = os.getenv('DB_DUCK_NAME','credit_rates.duckdb')
DB = DATA_DIR / DATA_URL
con = duckdb.connect(DB) # main connection to the database

* **Select a sample of the data**: 

In [4]:
with con.cursor() as cursor:
    cursor.execute("SELECT * FROM tasas_creditos ORDER BY fecha_corte LIMIT 5;")
    results = cursor.fetchall()
    for row in results:
        print(row)

(1, 'BC-ESTABLECIMIENTO BANCARIO', 13, 'BBVA Colombia', datetime.datetime(2025, 10, 3, 0, 0), 'Natural', 'Femenino', 'Microempresa', 'Comercial ordinario', 'Garantía fondo nacional de garantías (FNG)', 'Empresarial en pesos', 'Más de 1 año y hasta 3 años', 19.229999542236328, 0.0, 20000000.0, 1, 'Sin información (1)', 'más de 10 años', 'FS', 'Mayor a 12 SMLMV y menor o igual a 25 SMLMV', 'Deudor de la entidad', 5530, 5045, 'ANTIOQUIA', 'APARTADÓ')
(1, 'BC-ESTABLECIMIENTO BANCARIO', 60, 'Banco Mundo Mujer S.A.', datetime.datetime(2025, 10, 3, 0, 0), 'Natural', 'Masculino', 'Microempresa', 'Comercial ordinario', 'Garantía fondo nacional de garantías (FNG)', 'Empresarial en pesos', 'Más de 3 años y hasta 5 años', 24.360000610351562, 0.0, 26100000.0, 1, 'Sin información (1)', 'más de 10 años', 'FS', 'Mayor a 12 SMLMV y menor o igual a 25 SMLMV', 'Deudor de la entidad', 162, 50226, 'META', 'CUMARAL')
(1, 'BC-ESTABLECIMIENTO BANCARIO', 1, 'Banco de Bogotá', datetime.datetime(2025, 10, 3, 0, 

## **3. Initial Data Exploration and Sanity Checks**

* **Check data dimensions and data types**: 
    We will check the number of rows and columns in the dataset, as well as the data types of each column to ensure they are load appropriate for analysis.

In [11]:
with con.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) as total_rows FROM tasas_creditos;")
    print("Total rows in the dataset:", cursor.fetchone()[0])

Total rows in the dataset: 7968309


In [6]:
with con.cursor() as cursor:
    df_info = cursor.execute("DESCRIBE tasas_creditos;").df()
df_info

,column_name,column_type,null,key,default,extra
0,tipo_entidad,TINYINT,YES,None,None,None
1,nombre_tipo_entidad,VARCHAR,YES,None,None,None
2,codigo_entidad,SMALLINT,YES,None,None,None
3,nombre_entidad,VARCHAR,YES,None,None,None
4,fecha_corte,TIMESTAMP_NS,YES,None,None,None
5,tipo_de_persona,VARCHAR,YES,None,None,None
6,sexo,VARCHAR,YES,None,None,None
7,tama_o_de_empresa,VARCHAR,YES,None,None,None
8,tipo_de_cr_dito,VARCHAR,YES,None,None,None
9,tipo_de_garant_a,VARCHAR,YES,None,None,None


Is evident that all the columns have the correct data types. In addition, the dataset contains 24 columns and 7968309 rows.

* **Null values and missing data**